# ケプラーの法則を利用したシミュレーション

In [30]:
using JSON

global planets

# 元期軌道要素のデータを読み込む
function loadData(epoch)
    global planets
    file = JSON.parsefile("./data.json")
    planets = file[epoch]
end

# 時刻tにおける平均近点角Mを求める
function calcM(m0, t0, p, t)
    M = m0 + 2π * (t - t0) / p
    return M
end

# Mから離心近点角を求める
function calcE(M, e)
    E = M
    for i in 1:10
        E = E - (M - E + e * sin(E)) / (e * cos(E) - 1.0)
    end
    return E
end

# 天体の日心黄道座標を求める
function calcPosition(name, t)
    t0 = planets["epoch"]
    data = planets[name]
    incl = deg2rad(data["incl"])
    lan = deg2rad(data["lan"])
    peri = deg2rad(data["lperi"] - data["lan"])
    a = data["a"]
    e = data["e"]
    m0 = deg2rad(data["m0"])
    p = data["p"]

    E = calcE(calcM(m0,t0,p,t), e)

    # 軌道面上での座標(X,Y)を求める
    X = a * cos(E)-a*e
    Y = sqrt(1.0 - e^2) * a * sin(E)

    # 軌道面での座標を日心黄道座標に変換
    pos = [
        cos(lan) -sin(lan) 0.0
        sin(lan) cos(lan) 0.0
        0.0 0.0 1.0
    ] * [
        1.0 0.0
        0.0 cos(incl)
        0.0 sin(incl)
    ] * [
        cos(peri) -sin(peri)
        sin(peri) cos(peri)
    ] * [
        X
        Y
    ]
    return pos
end

function calcPosAndVel(name, t)
    t0 = planets["epoch"]
    data = planets[name]
    incl = deg2rad(data["incl"])
    lan = deg2rad(data["lan"])
    peri = deg2rad(data["lperi"] - data["lan"])
    a = data["a"]
    e = data["e"]
    m0 = deg2rad(data["m0"])
    p = data["p"]

    E = calcE(calcM(m0,t0,p,t), e)

    # 軌道面上での座標(X,Y)を求める
    X = a * cos(E)-a*e
    Y = sqrt(1.0 - e^2) * a * sin(E)

    # 万有引力定数
    G = 6.674e-11
    # 天文単位
    au = 1.495978e11
    # 太陽と惑星の質量の和(kg)
    u = G * (1.988e30 + 6.416e23)
    # 太陽との距離(m)
    r = sqrt(X^2+Y^2) * au
    # 速度ベクトルの傾き
    m = -sqrt(1.0 - e^2)/tan(E)
    # 軌道速度「軌道面上」(m/s)
    v = sqrt(u * (2/r - 1/(a * au)))

    # 軌道面上での速さ(VX,VY)を求める
    VX = v/sqrt(1+m^2)
    VY = v/sqrt(1+m^-2)

    # 軌道面系を日心黄道座標系に変換
    A = [
        cos(lan) -sin(lan) 0.0
        sin(lan) cos(lan) 0.0
        0.0 0.0 1.0
    ] * [
        1.0 0.0
        0.0 cos(incl)
        0.0 sin(incl)
    ] * [
        cos(peri) -sin(peri)
        sin(peri) cos(peri)
    ]

    pos = A * [
        X
        Y
    ]
    vel = A* [
        VX
        VY
    ]
    return pos, vel
end

loadData("2024-03-31")

# 2024/4/1
t = 2.4604015e6
println("pos[au], vel[m/s]")
for (key, value) in planets
    if key != "epoch"
        res = calcPosAndVel(key, t)
        printstyled(key, color=:cyan)
        println("\npos ", res[1])
        println("vel ", res[2])
    end
end

# 座標の単位はau

pos[au], vel[m/s]
jupiter
pos [2.9626994002744174, 4.030047128686638, -0.08297409352501968]
vel [5344.973724910137, 12470.28892752476, -171.25998178662002]
uranus
pos [11.743033241222665, 15.729006823002909, -0.09391005421133555]
vel [-6230.451428955788, -2322.2831273550323, 72.1919824091693]
neptune
pos [29.86717415924818, -1.8464644039820826, -0.6502075993807471]
vel [293.845447125615, 5450.432029989568, -118.99794107323308]
venus
pos [0.6353945017344536, -0.35276120082584195, -0.04150245586102198]
vel [-32208.76735788292, -13052.018544607568, 1679.050124792725]
mercury
pos [-0.33277261008821524, 0.12926196754357672, 0.04108896337351888]
vel [-6493.993379885762, 50796.30617056579, 4747.122426166178]
earth
pos [-0.9794548097881313, -0.19795165694211325, 1.4956955357880944e-5]
vel [-7995.668726222881, 28710.490042366873, -1.459292091278628]
mars
pos [0.9393652962242416, -1.0254610575824845, -0.04453588682426547]
vel [18780.930434211026, 18440.38268808071, -74.18055640321288]
saturn
pos

#### ユリウス日に変換

In [5]:
function to_jd(year, month, day)
    mjd = floor(year * 365.25) + floor(year/400) - floor(year/100) + floor(30.59 * (month - 2)) + day - 678912.0 + 2400000.5
    println(mjd)
end
to_jd(
    2024,
    4,
    1
)

2.4604015e6
